
Following tasks are undertaken:
* Columns Selection

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os

%matplotlib inline
from sklearn.model_selection import train_test_split
import warnings

warnings.filterwarnings("ignore")


In [3]:
_pipeline_name = 'redmodel'

_mh_root = os.path.join(os.environ['HOME'], 'redmodel')
_data_root = os.path.join(_mh_root, 'data', 'unprocessed')
# Python module file to inject customized logic into the TFX components. The
# Transform and Trainer both require user-defined functions to run successfully.
_module_file = os.path.join(_mh_root, 'taxi_utils.py')
# Path which can be listened to by the model server.  Pusher will output the
# trained model here.
_serving_model_dir = os.path.join(_mh_root, 'serving_model', _pipeline_name)

# Directory and data locations.  This example assumes all of the chicago taxi
# example code and metadata library is relative to $HOME, but you can store
# these files anywhere on your local filesystem.
_tfx_root = os.path.join(os.environ['HOME'], 'tfx')
_pipeline_root = os.path.join(_mh_root, 'pipelines', _pipeline_name)
# Sqlite ML-metadata db path.
_metadata_path = os.path.join(_mh_root, 'metadata', _pipeline_name,
                              'metadata.db')

In [6]:
# Data import
dreaddit_train_df = pd.read_csv(_data_root+"/dreaddit-train.csv")
dreaddit_test_df = pd.read_csv(_data_root+"/dreaddit-test.csv")
unsup_df = pd.read_csv(_data_root+"/unsup.csv")



In [7]:
def plot_graphs(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history["val_" + metric], "")
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend([metric, "val_" + metric])

In [8]:
dreaddit_train_df.head()

,subreddit,post_id,sentence_range,text,id,label,confidence,social_timestamp,social_karma,syntax_ari,...,lex_dal_min_pleasantness,lex_dal_min_activation,lex_dal_min_imagery,lex_dal_avg_activation,lex_dal_avg_imagery,lex_dal_avg_pleasantness,social_upvote_ratio,social_num_comments,syntax_fk_grade,sentiment
0,ptsd,8601tu,"(15, 20)","He said he had not felt that way before, sugge...",33181,1,0.8,1521614353,5,1.806818,...,1.000,1.1250,1.0,1.77000,1.52211,1.89556,0.86,1,3.253573,-0.002742
1,assistance,8lbrx9,"(0, 5)","Hey there r/assistance, Not sure if this is th...",2606,0,1.0,1527009817,4,9.429737,...,1.125,1.0000,1.0,1.69586,1.62045,1.88919,0.65,2,8.828316,0.292857
2,ptsd,9ch1zh,"(15, 20)",My mom then hit me with the newspaper and it s...,38816,1,0.8,1535935605,2,7.769821,...,1.000,1.1429,1.0,1.83088,1.58108,1.85828,0.67,0,7.841667,0.011894
3,relationships,7rorpp,"[5, 10]","until i met my new boyfriend, he is amazing, h...",239,1,0.6,1516429555,0,2.667798,...,1.000,1.1250,1.0,1.75356,1.52114,1.98848,0.50,5,4.104027,0.141671
4,survivorsofabuse,9p2gbc,"[0, 5]",October is Domestic Violence Awareness Month a...,1421,1,0.8,1539809005,24,7.554238,...,1.000,1.1250,1.0,1.77644,1.64872,1.81456,1.00,1,7.910952,-0.204167


In [9]:
print(dreaddit_test_df.text[5])

Thanks. Edit 1 - Fuel Receipt As Requested. <url> Sorry for the long responses, I went to spend the night at a friends because it got really cold here! The Police said they don't give out a copy of the report but they gave me an incident number that can be used to verify the report was filed.


In [12]:
cols = dreaddit_train_df.columns
cols

Index(['subreddit', 'post_id', 'sentence_range', 'text', 'id', 'label',
       'confidence', 'social_timestamp', 'social_karma', 'syntax_ari',
       ...
       'lex_dal_min_pleasantness', 'lex_dal_min_activation',
       'lex_dal_min_imagery', 'lex_dal_avg_activation', 'lex_dal_avg_imagery',
       'lex_dal_avg_pleasantness', 'social_upvote_ratio',
       'social_num_comments', 'syntax_fk_grade', 'sentiment'],
      dtype='object', length=116)

In [13]:
new_cols = [
    "text",
    "label",
]

In [14]:
train_df = dreaddit_train_df[new_cols]
train_df.head()

,text,label
0,"He said he had not felt that way before, sugge...",1
1,"Hey there r/assistance, Not sure if this is th...",0
2,My mom then hit me with the newspaper and it s...,1
3,"until i met my new boyfriend, he is amazing, h...",1
4,October is Domestic Violence Awareness Month a...,1


In [17]:
test_cols = dreaddit_test_df.columns
dreaddit_test_df.info()
test_df = dreaddit_test_df[new_cols]
test_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 715 entries, 0 to 714
Columns: 116 entries, id to sentiment
dtypes: float64(107), int64(5), object(4)
memory usage: 648.1+ KB


,text,label
0,"Its like that, if you want or not.“ ME: I have...",0
1,I man the front desk and my title is HR Custom...,0
2,We'd be saving so much money with this new hou...,1
3,"My ex used to shoot back with ""Do you want me ...",1
4,I haven’t said anything to him yet because I’m...,0


In [18]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2838 entries, 0 to 2837
Data columns (total 2 columns):
text     2838 non-null object
label    2838 non-null int64
dtypes: int64(1), object(1)
memory usage: 44.5+ KB


In [17]:
text_col = [
    "text"
]


In [15]:
unsup_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20050 entries, 0 to 20049
Data columns (total 2 columns):
Unnamed: 0    20050 non-null int64
text          20050 non-null object
dtypes: int64(1), object(1)
memory usage: 313.4+ KB


In [20]:
train_df.label.value_counts()

1    1488
0    1350
Name: label, dtype: int64

In [ ]:
train_df.to_csv(r"../data/final/train.csv")
test_df.to_csv(r"../data/final/eval/test.csv")
unsup_df.to_csv(r"../data/final/unsup.csv")